In [2]:
# TODO:

# 1.
# Create a summary for each of the six topics:
# For each topic, take the heading, title and all the headings AND titles of the all the articles of the topic's subsections.
# Each time a question is asked, ask LLM to provide the topic for the answer.
# NB! This costs 2x more than it could

# 2.
# Take the same from above and embed it so each topic's distance could be calculated for the Question.


# 3.
# Download the text for the articles and save these in a file. Have python create an automatic file structure.
# Embed all the articles and save the embeddings next to the article text.


In [37]:
import numpy as np

with open('embeddings/subsection_embeddings.npy', 'rb') as f:
    subsection_embeddings = np.load(f)

with open('embeddings/article_embeddings.npy', 'rb') as f:
    article_embeddings = np.load(f)

In [9]:
import openai
# openai.api_key = ''

In [38]:
EMBEDDING_MODEL = "text-embedding-ada-002"


def get_embedding(text: str, model: str = EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
        model=model,
        input=text
    )
    return result["data"][0]["embedding"]

In [63]:
question = 'Can I transfer moeny to a business in Ukraine?'
question_embedding = get_embedding(question)


In [64]:
import json

with open("scraped-data/index.json", "r") as f:
    data = json.load(f)

def return_content(article_id):
    counter = 0
    for section in data:
        for subsection in section["subsections"]:
            for article in subsection["articles"]:
                if counter != 241:
                    counter += 1
                    continue
                with open(article['folder_path'] + '/content.md', 'r') as f:
                    article_md = f.read()
                items = [section["title"], section["heading"], article_md]
                content = '\n\n'.join(items)
                return content, article['link']



In [65]:
import numpy as np


def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.

    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))


article_similarities = []

for i, article_embedding in enumerate(article_embeddings):
    article_similarities.append((vector_similarity(question_embedding, article_embedding), i))

article_similarities = sorted(article_similarities, reverse=True)

md, url = return_content(article_similarities[0][1])

print('wise.com' + url)

article_similarities[0][1]

wise.com/help/articles/2570950/are-there-any-restrictions-on-business-transfers


241

In [66]:
md

'Multi-user access, accounting and using our API.\n\nWise Business\n\n# Wise Business basics  \n## Are there any restrictions on business transfers?  \nWise is licensed to make personal transfers in all the countries we operate in. However in some cases, business transfers are slightly more limited. There are some currency routes which we can’t support business transfers to or from due to the way we’re regulated in those countries.\n\nFor the majority of our supported currencies there are no restrictions on Wise receiving money from your business. \n\nLearn more about each currency and their limitations by searching for "Guide to [currency]".\n\n### Currencies with business limitations\n\n\n--- CSV table begins ---  \nCurrency,Sending account can be a business,Recipient account can be a business,Additional note  \nAUD,Yes,Yes,See the business types we can support for Australia\xa0here.  \nBDT,No,No,Both the sender and the recipient of a transfer to BDT must be a privately owned account

In [67]:
prompt = '''
You are a customer support agent at Wise, the money transfer company. Your job is to answer the customer by using only the info provided here. You will get a customer's question on one line,
with the prefix "CUSTOMER: "
On the rest of the lines, you will get relevant information to answer the question. It starts with the prefix "CONTEXT: ".
Behave like you are the source of the information, while drawing answers only from the info provided. If your are unable to answer the question using the provided context, say 'I don't know'.
'''

In [68]:
COMPLETIONS_MODEL = "gpt-3.5-turbo-0301"

messages = [{"role": "system", "content": prompt}, {"role": "user", "content": 'CUSTOMER:  \n' + question}, {"role": "system", "content": 'CONTEXT: \n' + md},]
chat = openai.ChatCompletion.create(
    model=COMPLETIONS_MODEL,
    messages=messages
)
reply = chat.choices[0].message.content

reply

"Wise can only make transfers to private users of PrivatBank in Ukraine. The originating currency can arrive to us from a business or personal account, however we can't make transfers to any businesses in Ukraine."

In [69]:
messages.append({"role" : "assistant", "content":reply})
messages.append({"role": "user", "content": 'CUSTOMER:  \n' + 'How about Turkey?'})

chat = openai.ChatCompletion.create(
    model=COMPLETIONS_MODEL,
    messages=messages
)
reply = chat.choices[0].message.content

reply

'Wise can make bank transfers only from businesses that are registered in Turkey. The originating currency can arrive to us from a business or personal account and we can send transfers to both individuals and business bank accounts in Turkey.'